In [1]:
from bert import modeling

In [2]:
bert_config = modeling.BertConfig.from_json_file('BASE_config.json')

In [3]:
bert_config.__dict__

{'vocab_size': 32128,
 'hidden_size': 768,
 'num_hidden_layers': 12,
 'num_attention_heads': 12,
 'hidden_act': 'gelu',
 'intermediate_size': 3072,
 'hidden_dropout_prob': 0.1,
 'attention_probs_dropout_prob': 0.1,
 'max_position_embeddings': 1024,
 'type_vocab_size': 2,
 'initializer_range': 0.02,
 'directionality': 'bidi',
 'pooler_fc_size': 768,
 'pooler_num_attention_heads': 12,
 'pooler_num_fc_layers': 3,
 'pooler_size_per_head': 128,
 'pooler_type': 'first_token_transform'}

In [4]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [5]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [6]:
!rm -rf base
!mkdir base

In [7]:
files = [
'model.ckpt-175000.data-00000-of-00001',
'model.ckpt-175000.index',
'model.ckpt-175000.meta']

for file in files:
    blob = bucket.blob(f'b2b-base/{file}')
    blob = blob.download_to_filename(f'base/{file}')

In [8]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

In [9]:
bert_config = modeling.BertConfig.from_json_file('BASE_config.json')

In [10]:
import b2t

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = b2t.Model(bert_config, is_training = False)

sess.run(tf.global_variables_initializer())



Instructions for updating:
reduction_indices is deprecated, use axis instead



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use `tf.cast` instead.


Tensor("cls/predictions/MatMul:0", shape=(?, ?, 32128), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
saver = tf.train.Saver(var_list = tf.trainable_variables())
saver.restore(sess, 'base/model.ckpt-175000')

INFO:tensorflow:Restoring parameters from base/model.ckpt-175000


In [12]:
from prepro_utils import preprocess_text, encode_ids, encode_pieces
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.t5.model')

with open('sp10m.cased.t5.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [13]:
from unidecode import unidecode

def get_inputs(x):
    input_ids, input_masks, segment_ids = [], [], []
    for i in tqdm(range(len(x))):
        tokens_a = tokenizer.tokenize(unidecode(x[i]))
        tokens = tokens_a
        
        segment_id = [0] * len(tokens)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    return input_ids, input_masks, segment_ids

In [14]:
# https://harakahdaily.net/index.php/2020/05/22/apa-ph-buat-isu-kenaikan-harga-barang/

string2 = """
SEKARANG PH tengah mainkan isu harga barang untuk memburukkan Kerajaan PN. Kata Ahli Majlis Setiausaha PH (Saifuddin Nasution, Khalid Samad dan Ong Kian Ming) harga barang mentah dan basah menunjukkan ada peningkatan di banyak tempat. Ini membebankan rakyat kata mereka.

Saya setuju, Kerajaan PN perlu pantau dan ambil apa-apa tindakan yang perlu jika ada peningkatan harga barang basah atau mentah.

Tapi dalam masa yang sama saya fikir elok juga kalau kita imbas balik apa pula tindakan yang diambil oleh Kerajaan PH ketika rakyat tertekan dengan harga barang yang melonjak gila-gila masa mereka jadi Kerajaan Persekutuan selama 22 bulan dulu.

Mari kita semak balik.

(1) Kenaikan harga telur
Saifuddin Nasution – “Kementerian akan siasat.”
Salahuddin Ayub – “Ianya adalah perkara biasa disebabkan kenaikan bahan makanan ayam.”

(2) Kenaikan harga bawang 3 kali ganda
Saifuddin Nasution – “KPNHEP akan pantau kenaikan ini. Rakyat perlu tukar kepada bawang Holland.”

(3) Kenaikan harga ikan kembung
Salahuddin Ayub – “Saya pelik mengapa harga ikan kembung naik sedangkan ada 7 bilion ikan kembung dalam laut.”

(4) Kenaikan harga barangan kawalan
Saifuddin Nasution – “KPNHEP tidak menerima sebarang aduan kenaikan harga barang kawalan. Kita akan siasat.”

(5) Kenaikan harga barang mentah dan basah menjelang Aidilfitri 2019
Saifuddin Nasution – “KPNHEP puas hati dengan trend semasa harga barang keperluan. Tiada kenaikan mendadak (kenaikan sedikit demi sedikit itu perkara biasa).”

(6) Kenaikan harga barang keperluan pada penghujung 2018 dan sepanjang 2019
Lim Guan Eng – “Kenaikan harga barang disebakan banyak faktor seperti penyusutan nilai Ringgit, musim perayaan, kegiatan penyeludupan, kemarau di negara pengeluar selain sikap peniaga yang mahu mengaut keuntungan berlebihan. Ianya tidak dapat dielakkan.”

Yang saya listkan di atas hanya sebahagian kecil daripada kenyataan Menteri-menteri PH. Ada banyak lagi.

Kesimpulannya, PH tak buat apa-apa pun berkaitan kenaikan harga barang keperluan yang meresahkan rakyat semasa mereka jadi Kerajaan. Tapi sekarang tiba-tiba pula mereka jadi prihatin dan mahu jadi hero rakyat.
"""

In [15]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

# string = cleaning(string)
string2 = cleaning(string2)

In [16]:
input_ids, input_masks, segment_ids = get_inputs([f'ringkasan: {string2}'])

100%|██████████| 1/1 [00:00<00:00, 212.73it/s]


In [17]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

input_ids = pad_sequences(input_ids, padding='post')
input_masks = pad_sequences(input_masks, padding='post')
segment_ids = pad_sequences(segment_ids, padding='post')

In [18]:
r = sess.run(model.fast_result, feed_dict = {model.X: input_ids,
                                        model.segment_ids: segment_ids,
                                        model.input_masks: input_masks})

In [19]:
tokenizer.convert_ids_to_tokens(r[0].tolist())

['▁',
 '-',
 '▁Harga',
 '▁barang',
 '▁mentah',
 '▁dan',
 '▁basah',
 '▁menunjukkan',
 '▁ada',
 '▁peningkatan',
 '▁di',
 '▁banyak',
 '▁tempat',
 ',',
 '▁kata',
 '▁ahli',
 '▁Majlis',
 '▁Setiausaha',
 '▁PH',
 '.',
 '▁Harga',
 '▁barang',
 '▁mentah',
 '▁dan',
 '▁basah',
 '▁menunjukkan',
 '▁ada',
 '▁peningkatan',
 '▁di',
 '▁banyak',
 '▁tempat',
 ',',
 '▁lapor',
 '▁Ap',
 '.',
 '▁Ini',
 '▁membebankan',
 '▁rakyat',
 '▁kata',
 '▁mereka',
 '.',
 '▁',
 '"',
 'Saya',
 '▁setuju',
 ',',
 '▁Kerajaan',
 '▁',
 'PN',
 '▁perlu',
 '▁pantau',
 '▁dan',
 '▁mengambil',
 '▁apa',
 '-',
 'apa',
 '▁tindakan',
 '▁yang',
 '▁perlu',
 '▁jika',
 '▁ada',
 '▁peningkatan',
 '▁harga',
 '▁barang',
 '▁basah',
 '▁atau',
 '▁mentah',
 ',',
 '"',
 '▁kata',
 '▁seorang',
 '▁ahli',
 '▁Majlis',
 '▁Setiausaha',
 '▁PH',
 '.',
 '▁',
 '"',
 'Tetapi',
 '▁dalam',
 '▁masa',
 '▁yang',
 '▁sama',
 '▁saya',
 '▁fikir',
 '▁elok',
 '▁juga',
 '▁kalau',
 '▁kita',
 '▁imbas',
 '▁balik',
 '▁apa',
 '▁pula',
 '▁tindakan',
 '▁yang',
 '▁diambil',
 '▁oleh',

In [23]:
input_ids, input_masks, segment_ids = get_inputs([f'soalan: siapakah Mahathir Mohamad?'])
input_ids = pad_sequences(input_ids, padding='post')
input_masks = pad_sequences(input_masks, padding='post')
segment_ids = pad_sequences(segment_ids, padding='post')
r = sess.run(model.fast_result, feed_dict = {model.X: input_ids,
                                        model.segment_ids: segment_ids,
                                        model.input_masks: input_masks})

100%|██████████| 1/1 [00:00<00:00, 2375.03it/s]


In [24]:
tokenizer.convert_ids_to_tokens(r[0].tolist())

['▁Mahathir', '▁Mohamad', '</s>', '<pad>', '<pad>', '<pad>']

In [27]:
input_ids, input_masks, segment_ids = get_inputs([f'chatbot: ceritakan kepada saya tentang agama'])
input_ids = pad_sequences(input_ids, padding='post')
input_masks = pad_sequences(input_masks, padding='post')
segment_ids = pad_sequences(segment_ids, padding='post')
r = sess.run(model.fast_result, feed_dict = {model.X: input_ids,
                                        model.segment_ids: segment_ids,
                                        model.input_masks: input_masks})

tokenizer.convert_ids_to_tokens(r[0].tolist())

100%|██████████| 1/1 [00:00<00:00, 2157.56it/s]


['▁Kisah',
 '▁pertama',
 '▁dari',
 '▁agama',
 '▁Kristian',
 '▁yang',
 '▁telah',
 '▁meninggal',
 '▁dunia',
 '▁pada',
 '▁abad',
 '▁ke',
 '-',
 '1',
 '▁Masihi',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [28]:
!tar -czvf b2t.tar.gz base

base/
base/model.ckpt-175000.index
base/model.ckpt-175000.meta
base/model.ckpt-175000.data-00000-of-00001
